# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 70 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel70', 'models','t_dGP_train')

Linesearch     25;  Value 2.893257e+04


partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel70');
models = s.models;


## 1.1 RBCM vs TERBCM

### 1.1.4 Find multi q

In [5]:
criterion = 'TERBCM';

In [6]:
printf('Optimizing q\r\n');
iqs = 2.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.2079
   2.2029
   2.2003
   2.2052
   2.2078
   2.2070
   2.2008
   2.2096
   2.2011
   2.2083
   2.2021
   2.2035
   2.2002
   2.2062
   2.2029
   2.2012
   2.2026
   2.2044
   2.2051
   2.2020
   2.2056
   2.2025
   2.2052
   2.2071
   2.2013
   2.2008
   2.2027
   2.2032
   2.2057
   2.2028
   2.2088
   2.2039
   2.2047
   2.2085
   2.2085
   2.2071
   2.2028
   2.2068
   2.2072
   2.2034
   2.2037
   2.2002
   2.2057
   2.2045
   2.2084
   2.2029
   2.2095
   2.2002
   2.2095
   2.2055
   2.2098
   2.2064
   2.2051
   2.2014
   2.2029
   2.2082
   2.2062
   2.2041
   2.2040
   2.2012
   2.2079
   2.2017
   2.2079
   2.2089
   2.2085
   2.2034
   2.2026
   2.2012
   2.2039
   2.2087

ld =  100
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32

## 1.2 GRBCM vs TEGRBCM

### 1.2.4 Find multi q

In [7]:
criterion = 'TEGRBCM';

In [8]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0060
   2.0094
   2.0040
   2.0011
   2.0047
   2.0088
   2.0020
   2.0013
   2.0042
   2.0040
   2.0036
   2.0077
   2.0005
   2.0012
   2.0015
   2.0099
   2.0025
   2.0055
   2.0009
   2.0035
   2.0010
   2.0005
   2.0036
   2.0023
   2.0099
   2.0025
   2.0021
   2.0041
   2.0059
   2.0038
   2.0075
   2.0040
   2.0049
   2.0043
   2.0044
   2.0013
   2.0005
   2.0085
   2.0042
   2.0035
   2.0015
   2.0061
   2.0049
   2.0086
   2.0027
   2.0086
   2.0007
   2.0080
   2.0069
   2.0061
   2.0076
   2.0052
   2.0065
   2.0029
   2.0088
   2.0075
   2.0080
   2.0093
   2.0092
   2.0061
   2.0024
   2.0037
   2.0029
   2.0036
   2.0021
   2.0036
   2.0030
   2.0006
   2.0055
   2.0012

ld =  100
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32

## 1.3 GPoE vs TEGPoE

### 1.3.4 Find multi q

In [9]:
criterion = 'TEGPoE';

In [10]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0040
   2.0027
   2.0028
   2.0005
   2.0032
   2.0026
   2.0032
   2.0088
   2.0067
   2.0060
   2.0098
   2.0009
   2.0093
   2.0023
   2.0005
   2.0098
   2.0010
   2.0013
   2.0016
   2.0016
   2.0056
   2.0086
   2.0044
   2.0069
   2.0037
   2.0036
   2.0002
   2.0058
   2.0027
   2.0060
   2.0094
   2.0041
   2.0079
   2.0091
   2.0091
   2.0005
   2.0018
   2.0080
   2.0033
   2.0014
   2.0058
   2.0030
   2.0049
   2.0020
   2.0038
   2.0027
   2.0046
   2.0029
   2.0062
   2.0020
   2.0081
   2.0087
   2.0052
   2.0051
   2.0004
   2.0015
   2.0041
   2.0069
   2.0039
   2.0084
   2.0071
   2.0078
   2.0056
   2.0044
   2.0006
   2.0020
   2.0045
   2.0098
   2.0002
   2.0093

ld =  10
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
k =  31
k =  32
